# Deoxyfluorination with Sulfonyl Fluorides: Navigating Reaction Space with Machine Learning

DOI: 10.1021/jacs.8b01523

J. Am. Chem. Soc. 2018, 140, 5004−5008

## Defining protos for reaction data in Figure 1

Colab set-up: install schema

In [0]:
try:
  import ord_schema
except:
  !git clone https://github.com/Open-Reaction-Database/ord-schema.git
  %cd ord-schema
  !git pull
  !python setup.py build_py
  !python setup.py install

Import schema and helper functions

In [0]:
from datetime import datetime
from ord_schema.proto import reaction_pb2
from ord_schema.units import UnitResolver
from ord_schema.validations import validate_message
from ord_schema.message_helpers import build_compound, build_binary_data

unit_resolver = UnitResolver()

# Define exemplary reaction

This will be a single prototypical reaction from Table 1. We will copy it and use it as a template for filling out the full results table.

In [3]:
reaction = reaction_pb2.Reaction()
reaction.identifiers.add(value=r'deoxyfluorination', type='NAME')
reaction

identifiers {
  type: NAME
  value: "deoxyfluorination"
}

Define reaction inputs
- The first input is a pre-mixed stock solution of an alcohol in THF (1a for the prototype reaction)
- The second input is a pre-mixed stock solution of a sulfonyl fluoride (4-chlorobenzenesulfonyl fluoride for the prototype reaction)
- The third addition is a pure base (DBU for the prototype reaction)

In [0]:
# Input 1a: 0.1 mmol alcohol in 125 uL THF
input_1a = reaction.inputs['alcohol in THF']
input_1a.addition_order = 1

solute = input_1a.components.add()
solvent = input_1a.components.add()

solute.reaction_role = reaction_pb2.Compound.ReactionRole.REACTANT 
solute.identifiers.add(value=r'c1ccccc1CCC(O)C', type='SMILES')
solute.moles.CopyFrom(unit_resolver.resolve('0.1 mmol'))
solute.preparation.type = reaction_pb2.CompoundPreparation.PreparationType.NONE
solute.is_limiting = True

solvent.reaction_role = reaction_pb2.Compound.ReactionRole.SOLVENT
solvent.identifiers.add(value=r'THF', type='NAME')
solvent.identifiers.add(value=r'C1CCCO1', type='SMILES')
solvent.volume.CopyFrom(unit_resolver.resolve('125 uL'))
solvent.preparation.type = reaction_pb2.CompoundPreparation.PreparationType.DRIED

## Note: more lengthy way to specify volume w/o unit resolver
# solvent.volume.value = 125
# solvent.volume.units = reaction_pb2.Volume.MICROLITER

In [0]:
# Input sf: 0.11 mmol sulfonyl fluoride in 125 uL THF
input_sf = reaction.inputs['sulfonyl fluoride']
input_sf.addition_order = 2

solute = input_sf.components.add()
solvent = input_sf.components.add()

solute.reaction_role = reaction_pb2.Compound.ReactionRole.REACTANT 
solute.identifiers.add(value=r'4-chlorobenzenesulfonyl fluoride', type='NAME')
solute.identifiers.add(value=r'Clc1ccc(S=O(=O)F)cc1', type='SMILES')
solute.moles.CopyFrom(unit_resolver.resolve('0.11 mmol'))
solute.preparation.type = reaction_pb2.CompoundPreparation.PreparationType.SYNTHESIZED

solvent.reaction_role = reaction_pb2.Compound.ReactionRole.SOLVENT
solvent.identifiers.add(value=r'THF', type='NAME')
solvent.identifiers.add(value=r'C1CCCO1', type='SMILES')
solvent.volume.CopyFrom(unit_resolver.resolve('125 uL'))
solvent.preparation.type = reaction_pb2.CompoundPreparation.PreparationType.DRIED

In [0]:
# Input base: 0.15 mmol pure
input_base = reaction.inputs['base']
input_base.addition_order = 3
base = input_base.components.add()

base.reaction_role = reaction_pb2.Compound.ReactionRole.REAGENT
base.identifiers.add(value=r'N\2=C1\N(CCCCC1)CCC/2', type='SMILES')
base.moles.CopyFrom(unit_resolver.resolve('0.15 mmol'))
base.preparation.type = reaction_pb2.CompoundPreparation.PreparationType.NONE
base.vendor_source = 'Sigma-Millipore'
base.is_limiting = False

Define reaction setup & conditions

In [0]:
# Reactions performed in 1 mL glass vial
reaction.setup.vessel.CopyFrom(
    reaction_pb2.Vessel(
        type='VIAL', 
        material='GLASS',
        volume=unit_resolver.resolve('1 mL')
    )
)
reaction.setup.is_automated = False

In [0]:
# No temperature control = ambient conitions
t_conds = reaction.conditions.temperature
t_conds.type = t_conds.TemperatureControl.AMBIENT

In [0]:
# Vials were sealed under ambient conditions
p_conds = reaction.conditions.pressure
p_conds.type = p_conds.PressureControl.SEALED
p_conds.atmosphere = p_conds.Atmosphere.AIR

In [0]:
# Vials contained stir bars at 600 rpm
s_conds = reaction.conditions.stirring
s_conds.type = s_conds.StirringMethod.STIR_BAR
s_conds.rate = s_conds.StirringRate.HIGH # qualitative
s_conds.rpm = 600

No additional safety notes or observations

In [0]:
# No safety notes
reaction.notes.safety_notes = ''

# No reaction observations
# observation = reaction.observations.add()
# observation.time = 
# observation.comment = 

After 48 hours, the crude reaction mixture is characterized by 19F NMR to estimate the yield

This represents one "outcome" with one characterized "product"

In [0]:
outcome = reaction.outcomes.add()
outcome.reaction_time.CopyFrom(unit_resolver.resolve('48 hrs'))

# Analyses: 19F NMR
outcome.analyses['19f nmr of crude'].type = reaction_pb2.ReactionAnalysis.NMR
outcome.analyses['19f nmr of crude'].details = ('19F NMR using 1 equiv 1-fluoro'
  'naphthalene in 250 uL chloroform as internal standard')
outcome.analyses['19f nmr of crude'].instrument_manufacturer = 'Bruker'

# Define product identity
prod_2a = outcome.products.add() 
prod_2a.compound.identifiers.add().CopyFrom(
    reaction_pb2.CompoundIdentifier(value=r'c1ccccc1CCC(F)C', type='SMILES')
)
prod_2a.is_desired_product = True

# Define product yield from Figure 1 in the main test
# Note: described in SI, 4.8% is the variance of the yield including variance
# in the experiment itself. NMR yields itself has precision of <= 2%
# accoring to SI Section II.
prod_2a.compound_yield.CopyFrom(
    reaction_pb2.Percentage(value=40, precision=4.8) 
)

# The "19f nmr of crude" analysis was used to confirm both identity and yield
prod_2a.analysis_identity.append('19f nmr of crude')
prod_2a.analysis_yield.append('19f nmr of crude')

# Reaction provenance
reaction.provenance.city = r'Princeton, NJ'
reaction.provenance.doi = r'10.1021/jacs.8b01523'
reaction.provenance.publication_url = r'https://pubs.acs.org/doi/10.1021/jacs.8b01523'
reaction.provenance.record_created.time.value = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
reaction.provenance.record_created.person.CopyFrom(reaction_pb2.Person(
    name='Connor W. Coley', organization='MIT', orcid='0000-0002-8271-8723'
))

Validate and examine this final prototypical reaction entry -- note that this is just a single entry from the results table in Figure 1

In [13]:
reaction = validate_message(reaction)
print(reaction)

identifiers {
  type: NAME
  value: "deoxyfluorination"
}
inputs {
  key: "alcohol in THF"
  value {
    components {
      identifiers {
        type: SMILES
        value: "c1ccccc1CCC(O)C"
      }
      moles {
        value: 0.10000000149011612
        units: MILLIMOLES
      }
      reaction_role: REACTANT
      is_limiting: true
      preparation {
        type: NONE
      }
    }
    components {
      identifiers {
        type: NAME
        value: "THF"
      }
      identifiers {
        type: SMILES
        value: "C1CCCO1"
      }
      volume {
        value: 125.0
        units: MICROLITER
      }
      reaction_role: SOLVENT
      preparation {
        type: DRIED
      }
    }
    addition_order: 1
  }
}
inputs {
  key: "base"
  value {
    components {
      identifiers {
        type: SMILES
        value: "N\\2=C1\\N(CCCCC1)CCC/2"
      }
      moles {
        value: 0.15000000596046448
        units: MILLIMOLES
      }
      reaction_role: REAGENT
      preparation 

# Define full set of compounds and conditions

We will make extensive use of the ```build_compound``` helper function and define our own custom helper function to use the prototypical reaction example and replace only select entries

In [0]:
# Defining all major reactants, reagents, and products used in screen

compounds = {
  # Reactants
  '1a': build_compound(r'c1ccccc1CCC(O)C'),
  '2a': build_compound(r'c1ccccc1CCC(F)C'),
  '1b': build_compound(r'c1ccccc1-c2ccc(CO)cc2'),
  '2b': build_compound(r'c1ccccc1-c2ccc(CF)cc2'),
  '1c': build_compound(r'O[C@@H]1C[C@H](OCC2=CC=CC=C2)C1'),
  '2c': build_compound(r'F[C@@H]1C[C@H](OCC2=CC=CC=C2)C1'),
  '1d': build_compound(r'O[C@H]1C[C@@H](C(OC)=O)N(C(OC(C)(C)C)=O)C1'),
  '2d': build_compound(r'F[C@H]1C[C@@H](C(OC)=O)N(C(OC(C)(C)C)=O)C1'),
  # Sulfonyl fluorides
  '3-Cl': build_compound(r'Clc1ccc(S=O(=O)F)cc1'),
  'PyFluor': build_compound(r'O=S(C1=CC=CC=N1)(F)=O', name='PyFluor'),
  '3-CF3': build_compound(r'O=S(C1=CC=C(C(F)(F)F)C=C1)(F)=O'),
  '3-NO2': build_compound(r'O=S(C1=CC=C([N+]([O-])=O)C=C1)(F)=O'),
  'PBSF': build_compound(r'C(C(C(F)(F)S(=O)(=O)F)(F)F)(C(F)(F)F)(F)F', 
                         vendor='Acros'),
  # Bases
  'DBU': build_compound(r'N\2=C1\N(CCCCC1)CCC/2', name='DBU', 
                           vendor='Millipore-Sigma'),
  'MTBD': build_compound(r'CN1CCCN2C1=NCCC2', name='MTBD',
                            vendor='Sigma-Millipore'),
  'BTMG': build_compound(r'CC(C)(C)N=C(N(C)C)N(C)C', name='BTMG', 
                            vendor='Millipore-Sigma'),
  'BTPP': build_compound(r'CC(C)(C)N=P(N1CCCC1)(N2CCCC2)N3CCCC3', 
                            name='BTPP', vendor='Millipore-Sigma'),
}

In [0]:
# Define yield tables
data = [ # reactant, product, sulfonyl fluoride, base, yield percent
  # 2a
  ('1a', '2a', '3-Cl', 'DBU', 40),
  ('1a', '2a', '3-Cl', 'MTBD', 54),
  ('1a', '2a', '3-Cl', 'BTMG', 41),
  ('1a', '2a', '3-Cl', 'BTPP', 42),
  ('1a', '2a', 'PyFluor', 'DBU', 57),
  ('1a', '2a', 'PyFluor', 'MTBD', 59),
  ('1a', '2a', 'PyFluor', 'BTMG', 49),
  ('1a', '2a', 'PyFluor', 'BTPP', 53),
  ('1a', '2a', '3-CF3', 'DBU', 52),
  ('1a', '2a', '3-CF3', 'MTBD', 69),
  ('1a', '2a', '3-CF3', 'BTMG', 57),
  ('1a', '2a', '3-CF3', 'BTPP', 60),
  ('1a', '2a', '3-NO2', 'DBU', 54),
  ('1a', '2a', '3-NO2', 'MTBD', 63),
  ('1a', '2a', '3-NO2', 'BTMG', 55),
  ('1a', '2a', '3-NO2', 'BTPP', 51),
  ('1a', '2a', 'PBSF', 'DBU', 39),
  ('1a', '2a', 'PBSF', 'MTBD', 60),
  ('1a', '2a', 'PBSF', 'BTMG', 61),
  ('1a', '2a', 'PBSF', 'BTPP', 65),
  # 2b
  ('1b', '2b', '3-Cl', 'DBU', 11),
  ('1b', '2b', '3-Cl', 'MTBD', 36),
  ('1b', '2b', '3-Cl', 'BTMG', 83),
  ('1b', '2b', '3-Cl', 'BTPP', 92),
  ('1b', '2b', 'PyFluor', 'DBU', 12),
  ('1b', '2b', 'PyFluor', 'MTBD', 26),
  ('1b', '2b', 'PyFluor', 'BTMG', 57),
  ('1b', '2b', 'PyFluor', 'BTPP', 77),
  ('1b', '2b', '3-CF3', 'DBU', 17),
  ('1b', '2b', '3-CF3', 'MTBD', 36),
  ('1b', '2b', '3-CF3', 'BTMG', 83),
  ('1b', '2b', '3-CF3', 'BTPP', 99),
  ('1b', '2b', '3-NO2', 'DBU', 21),
  ('1b', '2b', '3-NO2', 'MTBD', 41),
  ('1b', '2b', '3-NO2', 'BTMG', 83),
  ('1b', '2b', '3-NO2', 'BTPP', 91),
  ('1b', '2b', 'PBSF', 'DBU', 23),
  ('1b', '2b', 'PBSF', 'MTBD', 37),
  ('1b', '2b', 'PBSF', 'BTMG', 48),
  ('1b', '2b', 'PBSF', 'BTPP', 68),
  # 2c
  ('1c', '2c', '3-Cl', 'DBU', 1),
  ('1c', '2c', '3-Cl', 'MTBD', 1),
  ('1c', '2c', '3-Cl', 'BTMG', 1),
  ('1c', '2c', '3-Cl', 'BTPP', 3),
  ('1c', '2c', 'PyFluor', 'DBU', 1),
  ('1c', '2c', 'PyFluor', 'MTBD', 2),
  ('1c', '2c', 'PyFluor', 'BTMG', 1),
  ('1c', '2c', 'PyFluor', 'BTPP', 1),
  ('1c', '2c', '3-CF3', 'DBU', 3),
  ('1c', '2c', '3-CF3', 'MTBD', 4),
  ('1c', '2c', '3-CF3', 'BTMG', 5),
  ('1c', '2c', '3-CF3', 'BTPP', 12),
  ('1c', '2c', '3-NO2', 'DBU', 11),
  ('1c', '2c', '3-NO2', 'MTBD', 12),
  ('1c', '2c', '3-NO2', 'BTMG', 12),
  ('1c', '2c', '3-NO2', 'BTPP', 22),
  ('1c', '2c', 'PBSF', 'DBU', 83),
  ('1c', '2c', 'PBSF', 'MTBD', 79),
  ('1c', '2c', 'PBSF', 'BTMG', 89),
  ('1c', '2c', 'PBSF', 'BTPP', 82),
  # 2d
  ('1d', '2d', '3-Cl', 'DBU', 25),
  ('1d', '2d', '3-Cl', 'MTBD', 30),
  ('1d', '2d', '3-Cl', 'BTMG', 29),
  ('1d', '2d', '3-Cl', 'BTPP', 30),
  ('1d', '2d', 'PyFluor', 'DBU', 45),
  ('1d', '2d', 'PyFluor', 'MTBD', 40),
  ('1d', '2d', 'PyFluor', 'BTMG', 39),
  ('1d', '2d', 'PyFluor', 'BTPP', 33),
  ('1d', '2d', '3-CF3', 'DBU', 41),
  ('1d', '2d', '3-CF3', 'MTBD', 47),
  ('1d', '2d', '3-CF3', 'BTMG', 39),
  ('1d', '2d', '3-CF3', 'BTPP', 33),
  ('1d', '2d', '3-NO2', 'DBU', 48),
  ('1d', '2d', '3-NO2', 'MTBD', 48),
  ('1d', '2d', '3-NO2', 'BTMG', 40),
  ('1d', '2d', '3-NO2', 'BTPP', 32),
  ('1d', '2d', 'PBSF', 'DBU', 59),
  ('1d', '2d', 'PBSF', 'MTBD', 74),
  ('1d', '2d', 'PBSF', 'BTMG', 64),
  ('1d', '2d', 'PBSF', 'BTPP', 58),
]

# Define all reactions in Table 1

The only aspects of reaction data that vary are (a) the identity of the solute of the ```'alcohol in THF'``` input, the identity of the solute in the ```'sulfonyl fluoride'``` input, and the identity of the sole component in the ```'base'``` input. The vessel, reaction setup, solvent, timing, temperature, etc. are all constant. The yield, of course, also changes, but the instrument and type of analysis (19F NMR) is constant.

In [0]:
reactions = []
for alc, prod, sf, base, y in data:
  
  # Start with template
  this_condition = reaction_pb2.Reaction()
  this_condition.CopyFrom(reaction)

  # Modify species:
  # - identifiers, vendor, prep ONLY
  def modify(cpd_from, cpd_to):
    del cpd_to.identifiers[:]
    for identifier in cpd_from.identifiers:
      cpd_to.identifiers.add().CopyFrom(identifier)
    cpd_to.vendor_source = cpd_from.vendor_source
    cpd_to.preparation.CopyFrom(cpd_from.preparation)
  
  modify(compounds[alc], this_condition.inputs['alcohol in THF'].components[0])
  modify(compounds[sf], this_condition.inputs['sulfonyl fluoride'].components[0])
  modify(compounds[base], this_condition.inputs['base'].components[0])
  modify(compounds[prod], this_condition.outcomes[0].products[0].compound)

  # Modify yield
  this_condition.outcomes[0].products[0].compound_yield.value = y

  # Validate
  this_condition = validate_message(this_condition)

  # Save
  reactions.append(this_condition)

In [17]:
print(f'Generated {len(reactions)} reactions')

Generated 80 reactions


In [19]:
# Inspect random reaction from this set
reactions[42]

identifiers {
  type: NAME
  value: "deoxyfluorination"
}
inputs {
  key: "alcohol in THF"
  value {
    components {
      identifiers {
        type: SMILES
        value: "O[C@@H]1C[C@H](OCC2=CC=CC=C2)C1"
      }
      moles {
        value: 0.10000000149011612
        units: MILLIMOLES
      }
      reaction_role: REACTANT
      is_limiting: true
      preparation {
      }
    }
    components {
      identifiers {
        type: NAME
        value: "THF"
      }
      identifiers {
        type: SMILES
        value: "C1CCCO1"
      }
      volume {
        value: 125.0
        units: MICROLITER
      }
      reaction_role: SOLVENT
      preparation {
        type: DRIED
      }
    }
    addition_order: 1
  }
}
inputs {
  key: "base"
  value {
    components {
      identifiers {
        type: SMILES
        value: "CC(C)(C)N=C(N(C)C)N(C)C"
      }
      identifiers {
        type: NAME
        value: "BTMG"
      }
      moles {
        value: 0.15000000596046448
        units: 

In [0]:
# import os
# if not os.path.isdir('example_protos'):
#   os.mkdir('example_protos')
# for i,reaction in enumerate(reactions):
#   with open(os.path.join('example_protos', f'ord-data-{i:03g}'), 'wb') as fid:
#     fid.write(reaction.SerializeToString())


In [0]:
# !zip -r example_protos.zip example_protos